
Facial Recognition using PyTorch and OpenCV

https://ritik12.medium.com/facial-recognition-using-pytorch-and-opencv-467c4e41d1f


Machine Learning - Face Recognition CNN Pytorch.ipynb
https://github.com/rubencg195/Pytorch-Tutorials/blob/master/Machine%20Learning%20-%20Face%20Recognition%20CNN%20Pytorch.ipynb



Face Recognition Using Pytorch
https://github.com/timesler/facenet-pytorch

Face Landmarks Detection With PyTorch

https://towardsdatascience.com/face-landmarks-detection-with-pytorch-4b4852f5e9c4



다중입력 deep neural network
https://rosenfelder.ai/multi-input-neural-network-pytorch/



Understanding dimensions in PyTorch
https://towardsdatascience.com/understanding-dimensions-in-pytorch-6edf9972d3be

In [17]:
from torch.optim import lr_scheduler
from torch.nn.init import *
from torchvision import transforms, utils, datasets, models
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pandas as pd
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torchvision import datasets
from torchvision.transforms import ToTensor

from FaceFeatureDataset import FaceFeatureDataset

# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")


Using cuda device


In [18]:
batch_size = 10
epochs = 10

In [19]:
linear_layer_2d = nn.Linear(in_features=68, out_features=128)
flatten = nn.Flatten()
input_2d = torch.randn(2, 2, 68) # n, channel, features
#print(input_2d)
print(input_2d.shape)
output_2d = linear_layer_2d(input_2d)
print('output_2d' , output_2d)
#print(output_2d.size())

#output_1d = flatten(input_2d)
#print(output_1d.size())
#linear_layer_2d(output_1d)
#print("1D ",  output_1d)

#test_sequence = nn.Sequential(nn.Linear(64, 32), nn.ReLU(), nn.Linear(32,10))
#out = test_sequence(input_2d)
#print(out)



torch.Size([2, 2, 68])
output_2d tensor([[[-3.1157e-01,  9.0461e-01, -2.4423e-01, -2.1884e-01, -9.1526e-01,
          -1.6587e-01,  1.0821e+00,  1.5103e-01,  4.5544e-01,  1.4594e-01,
          -7.0723e-01,  8.2956e-02, -6.6262e-01,  8.8002e-01,  3.4693e-01,
          -1.9007e-03, -1.3592e+00, -3.4829e-01, -6.0125e-01,  1.7638e-01,
          -6.6795e-01,  6.1491e-01, -7.3399e-01,  1.5171e-01, -6.4931e-01,
          -8.2073e-01, -9.7114e-01, -8.8151e-01,  3.2684e-01,  3.9353e-01,
          -3.6450e-01,  8.2247e-01, -1.6247e-01, -5.8801e-02,  4.9932e-02,
           5.3720e-01,  1.0799e+00, -6.2265e-01,  4.2779e-01,  4.9235e-01,
           4.4077e-01,  3.8501e-01, -7.9629e-01,  3.8377e-02, -4.9450e-01,
           7.3950e-01,  1.6512e-02,  7.8983e-01,  3.7818e-01,  9.0618e-01,
          -2.0608e-01, -4.9113e-01, -6.2648e-01, -7.4768e-01,  9.6963e-01,
          -3.2116e-01, -4.0987e-01, -5.9507e-01, -6.9109e-01,  1.0086e+00,
           1.2509e+00, -4.2139e-01,  3.8071e-01, -1.5466e+00, -1.30

In [20]:

testload = pd.read_csv("./outimg/Train/facefeature.csv")
landmarks = np.array(testload.iloc[0, 1:])
landmarks = landmarks.astype('float').reshape(-1, 2)
print(landmarks)

dataiter = iter(testload)
landmark = next(dataiter)
print(landmark)
# print(landmarks.shape)


[[-3.47850995e-01 -2.35903748e-01]
 [-3.41526145e-01 -1.45656640e-01]
 [-3.25950023e-01 -5.53465988e-02]
 [-3.08061083e-01  3.49791763e-02]
 [-2.83217955e-01  1.23039334e-01]
 [-2.37527996e-01  2.06615456e-01]
 [-1.73241092e-01  2.76440537e-01]
 [-9.49986118e-02  3.34795928e-01]
 [-1.41758445e-02  3.53849144e-01]
 [ 6.68514577e-02  3.42835724e-01]
 [ 1.48130495e-01  2.94817214e-01]
 [ 2.15642758e-01  2.30514577e-01]
 [ 2.71669597e-01  1.54569181e-01]
 [ 3.04646923e-01  6.69023600e-02]
 [ 3.28404442e-01 -2.54530310e-02]
 [ 3.52177696e-01 -1.20121240e-01]
 [ 3.64402591e-01 -2.17180935e-01]
 [-2.98935678e-01 -2.86455345e-01]
 [-2.52459047e-01 -3.18520130e-01]
 [-1.92278575e-01 -3.25049514e-01]
 [-1.36865340e-01 -3.10795002e-01]
 [-8.38593244e-02 -2.82679315e-01]
 [ 8.26950486e-02 -2.86172143e-01]
 [ 1.40688570e-01 -3.11219806e-01]
 [ 2.03213327e-01 -3.22359093e-01]
 [ 2.65627950e-01 -3.17308653e-01]
 [ 3.13929662e-01 -2.77660342e-01]
 [ 3.49282739e-03 -2.03445627e-01]
 [ 3.03655715e-03 -1

In [21]:
# Create DataLoader 

training_data = FaceFeatureDataset(feature_file="./outimg/Train/facefeature.csv", label_file="./Dataset/Train/csv/train.csv")
test_data = FaceFeatureDataset(feature_file="./outimg/Test/facefeature.csv", label_file="./Dataset/Test/csv/test.csv")

train_loader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

# 데이터 로드 확인 
for X, y in train_loader:
    print(f"Shape of X [N, F, C]: {X.shape}") # N , Channel, H= width W = height
    print(f"Shape of Tensor y: {y.shape} {y.dtype}")       
    break

n_total_steps = len(train_loader)
# print(f'Traing dat length {n_total_steps}')
# print(y)
print('X type', X.dtype)
print('y type', y.dtype)


Shape of X [N, F, C]: torch.Size([10, 68, 2])
Shape of Tensor y: torch.Size([10, 33]) torch.float32
X type torch.float32
y type torch.float32


In [22]:
#
#dataiter = iter(train_loader)
#landmark, labels = next(dataiter)
#print("landmark Shape ", landmark.shape)
#print(landmark)
#flatten = nn.Flatten()
#linear1 = nn.Linear(68, 32)
#x = flatten(landmark)
#print(x)
#print(x.size())
# print(landmark.shape)
# print('linear1', x.shape)


In [23]:

num_classes = 33

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(68 * 2, 32),
            nn.ReLU(),
            nn.Linear(32, num_classes),
            nn.ReLU(),
        )
    

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)


NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=136, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=33, bias=True)
    (3): ReLU()
  )
)


In [24]:
# criterion = nn.CrossEntropyLoss()
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)


In [25]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # Compute prediction error
        pred = model(X)
        # print(pred)
        loss = loss_fn(pred, y)
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # print('batch',  batch)
        if batch % 10 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [26]:
# Training
epochs = 200

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_loader, model, criterion, optimizer)    
print("Done!")

Epoch 1
-------------------------------
loss: 0.233010  [    0/  100]
Epoch 2
-------------------------------
loss: 0.216255  [    0/  100]
Epoch 3
-------------------------------
loss: 0.225929  [    0/  100]
Epoch 4
-------------------------------
loss: 0.232366  [    0/  100]
Epoch 5
-------------------------------
loss: 0.229129  [    0/  100]
Epoch 6
-------------------------------
loss: 0.219246  [    0/  100]
Epoch 7
-------------------------------
loss: 0.232491  [    0/  100]
Epoch 8
-------------------------------
loss: 0.218484  [    0/  100]
Epoch 9
-------------------------------
loss: 0.231237  [    0/  100]
Epoch 10
-------------------------------
loss: 0.237416  [    0/  100]
Epoch 11
-------------------------------
loss: 0.243295  [    0/  100]
Epoch 12
-------------------------------
loss: 0.207508  [    0/  100]
Epoch 13
-------------------------------
loss: 0.241346  [    0/  100]
Epoch 14
-------------------------------
loss: 0.223628  [    0/  100]
Epoch 15
------

In [27]:
PATH = './cifar_net.pth'
torch.save(model.state_dict(), PATH)

In [28]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    print('test size', size )
    # num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0,0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            loss = loss_fn(pred, y)            
            print('pred =', pred)
            #print('loss', loss)
            #print('real', y)



In [29]:

test(test_loader, model, criterion)

print("Done !")

test size 100
pred = tensor([[0.0000, 0.2925, 0.2273, 0.0000, 0.2239, 0.0000, 0.1286, 0.1859, 0.3306,
         0.1947, 0.4036, 0.0000, 0.0000, 0.0000, 0.2584, 0.1199, 0.0000, 0.2126,
         0.0000, 0.1994, 0.1438, 0.0000, 0.1357, 0.2535, 0.1694, 0.0000, 0.1264,
         0.0000, 0.0000, 0.1993, 0.0000, 0.1827, 0.1842],
        [0.0000, 0.2956, 0.2309, 0.0000, 0.2246, 0.0000, 0.1271, 0.1859, 0.3324,
         0.1936, 0.4054, 0.0000, 0.0000, 0.0000, 0.2544, 0.1177, 0.0000, 0.2116,
         0.0000, 0.2039, 0.1441, 0.0000, 0.1381, 0.2554, 0.1680, 0.0000, 0.1258,
         0.0000, 0.0000, 0.2006, 0.0000, 0.1813, 0.1844],
        [0.0000, 0.2944, 0.2292, 0.0000, 0.2248, 0.0000, 0.1283, 0.1855, 0.3303,
         0.1919, 0.4042, 0.0000, 0.0000, 0.0000, 0.2574, 0.1164, 0.0000, 0.2114,
         0.0000, 0.2013, 0.1436, 0.0000, 0.1383, 0.2536, 0.1687, 0.0000, 0.1258,
         0.0000, 0.0000, 0.1999, 0.0000, 0.1823, 0.1834],
        [0.0000, 0.2950, 0.2309, 0.0000, 0.2248, 0.0000, 0.1289, 0.1830, 0.3